In [ ]:

%load_ext autoreload
%autoreload 2



In [ ]:
# default_exp core


 # module name here

 > API details.

In [ ]:
#hide
from nbdev.showdoc import *



In [ ]:
from fastai2.tabular.all import *
from fastai2.vision.all import *
from fastai2.data.load import _FakeLoader, _loaders
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import datetime



In [ ]:
from fastai2.data.load import _FakeLoader, _loaders
class MixedDL():
    def __init__(self, tab_dl:TabDataLoader, vis_dl:TfmdDL, device='cuda:0'):
        "Stores away `tab_dl` and `vis_dl`, and overrides `shuffle_fn`"
        self.device = device
        tab_dl.shuffle_fn = self.shuffle_fn
        vis_dl.shuffle_fn = self.shuffle_fn
        self.dls = [tab_dl, vis_dl]
        self.count = 0
        self.fake_l = _FakeLoader(self, False, 0, 0)



In [ ]:
@patch
def shuffle_fn(x:MixedDL, idxs):
        "Generates a new `rng` based upon which `DataLoader` is called"
        if x.count == 0: # if we haven't generated an rng yet
            x.rng = x.dls[0].rng.sample(idxs, len(idxs))
            x.count += 1
            return x.rng
        else:
            x.count = 0
            return x.rng

@patch
def __iter__(dl:MixedDL):
        "Iterate over your `DataLoader`"
        z = zip(*[_loaders[i.fake_l.num_workers==0](i.fake_l) for i in dl.dls])
        for b in z:
            if dl.device is not None:
                b = to_device(b, dl.device)
            batch = []
            batch.extend(dl.dls[0].after_batch(b[0])[:2]) # tabular cat and cont
            batch.append(dl.dls[1].after_batch(b[1][0])) # Image
            batch.append(b[1][1]) # y
            yield tuple(batch)

@patch
def one_batch(x:MixedDL):
        "Grab a batch from the `DataLoader`"
        with x.fake_l.no_multiproc(): res = first(x)
        if hasattr(x, 'it'): delattr(x, 'it')
        return res

@patch
def __len__(x:MixedDL): return len(x.dls[0])

@patch
def to(x:MixedDL, device): x.device = device

@patch
def show_batch(x:MixedDL):
    "Show a batch from multiple `DataLoaders`"
    for dl in x.dls:
        dl.show_batch()

@patch
def show_results(x:MixedDL,b,out,**kwargs):
    "Show a batch from multiple `DataLoaders`"
    for i,dl in enumerate(x.dls):
        if i == 0:
            dl.show_results(b=b[:2]+(b[3],),out=out,**kwargs)
        else:
            dl.show_results(b=b[2:],out=out,**kwargs)

@patch
def new(x:MixedDL,*args,**kwargs):
    "Show a batch from multiple `DataLoaders`"
    new_dls = [dl.new(*args,**kwargs) for dl in x.dls]
    res=MixedDL(*new_dls)
    return res



In [ ]:
path = "/home/marcel/projetos/data/csgo_analyze/processed_test/de_mirage"
image_files = get_image_files(path)
tabular_files = get_files(path, extensions=['.csv'])
print(len(image_files))
print(len(tabular_files))
bs=64


10144
278


In [ ]:
def fileLabeller(o,**kwargs):
    winnerFile = Path(o).parent/"winner.txt"
    if os.path.isfile(winnerFile):
        f = open(winnerFile, "r")
        winner = f.readline()
        f.close()
    else:
        winner="na"
    return winner

print(image_files[0])
print(fileLabeller(image_files[0]))



/home/marcel/projetos/data/csgo_analyze/processed_test/de_mirage/6/ct_2_t_3/output_map16.jpg
ct


In [ ]:
columns = ["t_1","t_2","t_3","t_4","t_5","ct_1","ct_2","ct_3","ct_4","ct_5",
    "t_1_blindtime", "t_2_blindtime", "t_3_blindtime", "t_4_blindtime", "t_5_blindtime",
	"ct_1_blindtime", "ct_2_blindtime", "ct_3_blindtime", "ct_4_blindtime", "ct_5_blindtime",
    "t_1_mainweapon", "t_1_secweapon", "t_1_flashbangs", "t_1_hassmoke", "t_1_hasmolotov", "t_1_hashe", "t_1_armor", "t_1_hashelmet", "t_1_hasc4",
		"t_2_mainweapon", "t_2_secweapon", "t_2_flashbangs", "t_2_hassmoke", "t_2_hasmolotov", "t_2_hashe", "t_2_armor", "t_2_hashelmet", "t_2_hasc4",
		"t_3_mainweapon", "t_3_secweapon", "t_3_flashbangs", "t_3_hassmoke", "t_3_hasmolotov", "t_3_hashe", "t_3_armor", "t_3_hashelmet", "t_3_hasc4",
		"t_4_mainweapon", "t_4_secweapon", "t_4_flashbangs", "t_4_hassmoke", "t_4_hasmolotov", "t_4_hashe", "t_4_armor", "t_4_hashelmet", "t_4_hasc4",
		"t_5_mainweapon", "t_5_secweapon", "t_5_flashbangs", "t_5_hassmoke", "t_5_hasmolotov", "t_5_hashe", "t_5_armor", "t_5_hashelmet", "t_5_hasc4",
		"ct_1_mainweapon", "ct_1_secweapon", "ct_1_flashbangs", "ct_1_hassmoke", "ct_1_hasmolotov", "ct_1_hashe", "ct_1_armor", "ct_1_hashelmet", "ct_1_hasdefusekit",
		"ct_2_mainweapon", "ct_2_secweapon", "ct_2_flashbangs", "ct_2_hassmoke", "ct_2_hasmolotov", "ct_2_hashe", "ct_2_armor", "ct_2_hashelmet", "ct_2_hasdefusekit",
		"ct_3_mainweapon", "ct_3_secweapon", "ct_3_flashbangs", "ct_3_hassmoke", "ct_3_hasmolotov", "ct_3_hashe", "ct_3_armor", "ct_3_hashelmet", "ct_3_hasdefusekit",
		"ct_4_mainweapon", "ct_4_secweapon", "ct_4_flashbangs", "ct_4_hassmoke", "ct_4_hasmolotov", "ct_4_hashe", "ct_4_armor", "ct_4_hashelmet", "ct_4_hasdefusekit",
		"ct_5_mainweapon", "ct_5_secweapon", "ct_5_flashbangs", "ct_5_hassmoke", "ct_5_hasmolotov", "ct_5_hashe", "ct_5_armor", "ct_5_hashelmet", "ct_5_hasdefusekit",
        "round_time",
        'related_image','winner']
full_csv = pd.DataFrame(columns=columns)
for i,tab_file in enumerate(tabular_files):
    if i%50==0:
        print("processing file:",i,"of",len(tabular_files))
    if not os.stat(tab_file).st_size == 0 and os.path.isfile(tab_file.parent/"winner.txt"):
        new_csv = pd.read_csv(tab_file)
        new_csv['index']=new_csv.index
        new_csv['related_image'] = str(tab_file.parent)+"/output_map"+new_csv['index'].astype(str)+".jpg"
        new_csv['winner'] = fileLabeller(tab_file)
        new_csv=new_csv.drop(columns=["index"])
        new_csv.columns=columns
        full_csv=full_csv.append(new_csv)



processing file: 0 of 278
processing file: 50 of 278
processing file: 100 of 278
processing file: 150 of 278
processing file: 200 of 278
processing file: 250 of 278


In [ ]:
filtered_image_files = L()
for image_file in image_files:
    if fileLabeller(image_file) in ["t","ct"] and not os.stat(image_file.parent/"tabular.csv").st_size == 0 and str(image_file) in full_csv['related_image'].values:
        filtered_image_files.append(image_file)



In [ ]:
def roundSplitter(filtered_image_files):
    uniqueList = list(set([Path(o).parent for o in filtered_image_files]))
    splits=RandomSplitter()(uniqueList)
    train_image_files=L()
    valid_image_files=L()
    for i,o in enumerate(filtered_image_files):
        if uniqueList.index(Path(o).parent) in splits[0]:
            train_image_files+=i
        else:
            valid_image_files+=i
    return train_image_files,valid_image_files



In [ ]:
filtered_image_files.sort()
full_csv=full_csv.sort_values(by=['related_image'])
#filtered_image_files=filtered_image_files[:2000]
#full_csv=full_csv.iloc[:2000,:]
splits=roundSplitter(filtered_image_files)



In [ ]:
print(filtered_image_files[55])
full_csv.iloc[55,:]



/home/marcel/projetos/data/csgo_analyze/processed_test/de_mirage/10/ct_0_t_1/output_map59.jpg


t_1                                                                                                           0.65
t_2                                                                                                              0
t_3                                                                                                           0.37
t_4                                                                                                              0
t_5                                                                                                              0
                                                                 ...                                              
ct_5_hashelmet                                                                                                   0
ct_5_hasdefusekit                                                                                                0
round_time                                                                      

In [ ]:
cont_names = ['t_1', 't_2','t_3','t_4','t_5',
    'ct_1','ct_2','ct_3','ct_4','ct_5',
    "t_1_blindtime", "t_2_blindtime", "t_3_blindtime", "t_4_blindtime", "t_5_blindtime",
	"ct_1_blindtime", "ct_2_blindtime", "ct_3_blindtime", "ct_4_blindtime", "ct_5_blindtime",
    "t_1_armor","t_2_armor","t_3_armor","t_4_armor","t_5_armor",
    "ct_1_armor","ct_2_armor","ct_3_armor","ct_4_armor","ct_5_armor","round_time"]
cat_names = ["t_1_mainweapon", "t_1_secweapon", "t_1_flashbangs", "t_1_hassmoke", "t_1_hasmolotov", "t_1_hashe", "t_1_hashelmet", "t_1_hasc4",
		"t_2_mainweapon", "t_2_secweapon", "t_2_flashbangs", "t_2_hassmoke", "t_2_hasmolotov", "t_2_hashe", "t_2_hashelmet", "t_2_hasc4",
		"t_3_mainweapon", "t_3_secweapon", "t_3_flashbangs", "t_3_hassmoke", "t_3_hasmolotov", "t_3_hashe", "t_3_hashelmet", "t_3_hasc4",
		"t_4_mainweapon", "t_4_secweapon", "t_4_flashbangs", "t_4_hassmoke", "t_4_hasmolotov", "t_4_hashe", "t_4_hashelmet", "t_4_hasc4",
		"t_5_mainweapon", "t_5_secweapon", "t_5_flashbangs", "t_5_hassmoke", "t_5_hasmolotov", "t_5_hashe", "t_5_hashelmet", "t_5_hasc4",
		"ct_1_mainweapon", "ct_1_secweapon", "ct_1_flashbangs", "ct_1_hassmoke", "ct_1_hasmolotov", "ct_1_hashe", "ct_1_hashelmet", "ct_1_hasdefusekit",
		"ct_2_mainweapon", "ct_2_secweapon", "ct_2_flashbangs", "ct_2_hassmoke", "ct_2_hasmolotov", "ct_2_hashe", "ct_2_hashelmet", "ct_2_hasdefusekit",
		"ct_3_mainweapon", "ct_3_secweapon", "ct_3_flashbangs", "ct_3_hassmoke", "ct_3_hasmolotov", "ct_3_hashe", "ct_3_hashelmet", "ct_3_hasdefusekit",
		"ct_4_mainweapon", "ct_4_secweapon", "ct_4_flashbangs", "ct_4_hassmoke", "ct_4_hasmolotov", "ct_4_hashe", "ct_4_hashelmet", "ct_4_hasdefusekit",
		"ct_5_mainweapon", "ct_5_secweapon", "ct_5_flashbangs", "ct_5_hassmoke", "ct_5_hasmolotov", "ct_5_hashe", "ct_5_hashelmet", "ct_5_hasdefusekit"]
for cat in cont_names:
    full_csv[cat]=full_csv[cat].astype(np.float)
procs = [Categorify, Normalize]
dls_tabular = TabularDataLoaders.from_df(full_csv, path, procs=procs, cont_names=cont_names,cat_names=cat_names,
                                 y_names="winner", bs=bs,valid_idx=splits[1],device=torch.device('cuda:0'))



In [ ]:

dsets = Datasets(filtered_image_files, [[PILImage.create], [fileLabeller, Categorize]],splits=splits)
item_tfms = [Resize(200),ToTensor]
batch_tfms = [IntToFloatTensor, Normalize.from_stats(*imagenet_stats)]
dls_image = dsets.dataloaders(after_item=item_tfms, after_batch=batch_tfms, bs=bs, num_workers=8,device=torch.device("cuda"))



In [ ]:
train_mixed_dl = MixedDL(dls_tabular.train, dls_image.train)
train_mixed_dl.to(torch.device('cuda:0'))
valid_mixed_dl = MixedDL(dls_tabular.valid,dls_image.valid)
valid_mixed_dl.to(torch.device('cuda:0'))
dls = DataLoaders(train_mixed_dl, valid_mixed_dl,device=torch.device('cuda:0'))



In [ ]:
dls_image.one_batch()



(TensorImage([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-2.1179, -2.1179, -2.1179,  ..., -2.1008, -2.0665, -2.1008],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1008, -2.0837, -2.1008],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.0837, -2.1008]],
 
          [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           ...,
           [-2.0357, -2.0357, -2.0357,  ..., -2.0182, -1.9832, -2.0182],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0182, -2.0007, -2.0182],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0007, -2.0182]],
 
          [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
        

In [ ]:
breakpoint()
dls_tabular.one_batch()
tabular_learner(dls_tabular)
